# Redes neuronales

#### Red neuronal de 2 capas

<img src="images/neural_networks.jpeg" alt="Drawing" style="width: 500px;"/>

El descenso de gradiente se puede utilizar para encontrar el mejor $W$ y $b$. El enfoque es básicamente el mismo como vimos anteriomente.

**Paso 0: Elegir función de activación**


<img src="images/activation_functions.png" alt="Drawing" style="width: 800px;"/>


- La función de activación elegida tiene gran peso en la capacidad de la red neuronal.
- Las capas internas suelen utilizar la misma función de activación y la capa de salida suele tener una función de activación diferente y depende del tipo de problema.
- Las funciones de activación usualmente son diferenciables (cálculo de gradientes). Hay casos donde NO (RELU).

¿Por que necesitamos funciones de activacion?

**Paso 1: Inicializar pesos**

<img src="images/entrada_ponderada.png" alt="Drawing" style="width: 150px;"/>


* Inicializar pesos $W$ con valores aleatorios distintos de cero. $b$ > 0

**Paso 2: Propagación hacia adelante**

<img src="images/forward_propagation.png" alt="Drawing" style="width: 170px;"/>


**Paso 3: Calcular costo**


<img src="images/cross_entropy.png" alt="Drawing" style="width: 600px;"/>


* La función de costo a utilizar depende del problema a resolver, pero suponiendo un problema de clasificación binaria, podemos utilizar la función de entropia cruzada.

Llamemos $J$ a nuestra función de costo.

<img src="images/bce.png" alt="Drawing" style="width: 400px;"/>


**Paso 4: Propagación hacia atrás**

La retropropagación comienza tomando la derivada parcial de la función de costo

Última capa:

<img src="images/retro_2.png" alt="Drawing" style="width: 230px;"/>


Primera capa:

<img src="images/retro_1.png" alt="Drawing" style="width: 250px;"/>


Esto se puede generalizar a N capas

**Paso 5: Actualización de pesos**

<img src="images/act_pesos.png" alt="Drawing" style="width: 230px;"/>

# Redes neuronales con PyTorch

Las redes de aprendizaje profundo tienden a ser masivas con docenas o cientos de capas, de ahí proviene el término "profundo". 
Construir redes neuronales desde cero es bastante engorroso y difícil.
PyTorch tiene un módulo nn que proporciona una buena forma de construir de forma eficiente grandes redes neuronales.

## XOR 

In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('xor.csv')
X = df[['x1', 'x2']].values
y = df['class label'].values

In [ ]:
RANDOM_SEED = 124
DEVICE = ('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
plt.scatter(X[y==0, 0], X[y==0, 1], marker='o')
plt.scatter(X[y==1, 0], X[y==1, 1], marker='s')

plt.tight_layout()
#plt.savefig('xor.pdf')
plt.show()

In [ ]:
class MLPLinear(torch.nn.Module):

    def __init__(self, num_features, num_hidden_1, num_classes):
        super(MLPLinear, self).__init__()
        
        self.num_classes = num_classes
        
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_out = torch.nn.Linear(num_hidden_1, num_classes)
        
    def forward(self, x):
        
        out = self.linear_1(x)
        
        output = self.linear_out(out)
        prob = F.softmax(output, dim=1)
        return output, prob

In [ ]:
torch.manual_seed(RANDOM_SEED)

model1 = MLPLinear(num_features=2,
                   num_hidden_1=50,
                   num_classes=2)

model1 = model1.to(DEVICE)

optimizer = torch.optim.SGD(model1.parameters(), lr=0.1)

In [ ]:
start_time = time.time()
minibatch_cost = []

NUM_EPOCHS = 25

features = torch.tensor(X, dtype=torch.float).to(DEVICE)
targets = torch.tensor(y, dtype=torch.long).to(DEVICE)

for epoch in range(NUM_EPOCHS):

    logits, probas = model1(features)

    cost = F.cross_entropy(logits, targets)
    optimizer.zero_grad()

    cost.backward()
    minibatch_cost.append(cost)

    optimizer.step()

    ### LOGGING

    print (f'Epoca: {epoch+1:03d}/{NUM_EPOCHS:03d} | Coste: {cost:.4f}')

    
print('Tiempo de entrenamiento total: %.2f min' % ((time.time() - start_time)/60))

In [ ]:
from matplotlib.colors import ListedColormap
import numpy as np


def plot_decision_regions(X, y, classifier, resolution=0.02):

    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    tensor = torch.tensor(np.array([xx1.ravel(), xx2.ravel()]).T).float()
    logits, probas = classifier.forward(tensor)
    Z = np.argmax(probas.detach().numpy(), axis=1)

    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, color=cmap(idx),
                    edgecolor='black',
                    marker=markers[idx], 
                    label=cl)

In [ ]:
plot_decision_regions(features, targets, classifier=model1)

plt.tight_layout()

plt.show()

### Cambiando la funcion de activacion

In [ ]:
class MLPReLU(torch.nn.Module):

    def __init__(self, num_features, num_hidden_1, num_classes):
        super(MLPReLU, self).__init__()
        
        self.num_classes = num_classes
        
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_classes)
        self.linear_out
        
        
    def forward(self, x):
        
        out = self.linear_1(x)
        out = F.relu(out)
        
        out = self.linear_2()
        
        output = self.linear_out(out)
        prob = F.softmax(output, dim=1)
        return output, prob

In [ ]:
torch.manual_seed(RANDOM_SEED)

model2 = MLPReLU(num_features=2,
                num_hidden_1=50,
                num_classes=2)

model2 = model2.to(DEVICE)

optimizer = torch.optim.SGD(model2.parameters(), lr=0.1)

In [ ]:
start_time = time.time()
minibatch_cost = []

NUM_EPOCHS = 25

features = torch.tensor(X, dtype=torch.float).to(DEVICE)
targets = torch.tensor(y, dtype=torch.long).to(DEVICE)

for epoch in range(NUM_EPOCHS):

    logits, probas = model2(features)

    cost = F.cross_entropy(logits, targets)
    optimizer.zero_grad()

    cost.backward()
    minibatch_cost.append(cost)

    optimizer.step()


    print (f'Epoca: {epoch+1:03d}/{NUM_EPOCHS:03d} | Coste: {cost:.4f}')

    
print('Tiempo total de entrenamiento: %.2f min' % ((time.time() - start_time)/60))

In [ ]:
plot_decision_regions(features, targets, classifier=model2)

plt.tight_layout()
plt.show()

### Imagenes: Obtener dataset

In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
batch_size = 10

# el proceso principal es el encargado de cargar cada batch 
# cuidado: mas workers incrementa el uso de memoria
num_workers = 0  

# convertir datos a formato torch tensor
transform = transforms.ToTensor()

# dataset de entrenamiento
train_data = datasets.MNIST(
    root='data', train=True, download=True, transform=transform
)

# dataset de test
test_data = datasets.MNIST(
    root='data', train=False, download=True, transform=transform
)


# dataloaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=batch_size, num_workers=num_workers
)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=batch_size, num_workers=num_workers
)

## Visualizando un batch

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
    
# obtener un batch del dataset de entrenamiento
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

# plot de las imagenes del dataset con su etiqueta (anotación/label) en el título
fig = plt.figure(figsize=(25, 7))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title(str(labels[idx].item()), fontsize=20)

In [ ]:
img = np.squeeze(images[5])

fig = plt.figure(figsize = (12, 12)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')
width, height = img.shape
thresh = img.max()/2.5
for x in range(width):
    for y in range(height):
        val = round(img[x][y],2) if img[x][y] !=0 else 0
        ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center',
                    color='white' if img[x][y]<thresh else 'black')


## Definiendo la arquitectura de la red

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

model = Net()
print(model)

In [ ]:
# especificar funcion de perdida
criterion = nn.CrossEntropyLoss()

# algoritmo de optimizacion: descenso del gradiente
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


In [ ]:
# cantidad de epocas a entrenar el modelo
n_epochs = 2

# habilitar modelo para training (actualizacion de gradientes)
model.train()

for epoch in range(n_epochs):
    # para monitorear el loss 
    train_loss = 0.0
    
    for data, target in train_loader:
        # borrar gradientes
        optimizer.zero_grad()
        
        # propagacion hacia adelante
        output = model(data)
        
        # calcular salida de funcion de perdida
        loss = criterion(output, target)
        
        # propagacion hacia atras
        loss.backward()
        
        # actualizacion de parametros
        optimizer.step()
        
        # actualizar valor de loss
        train_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoca: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))

In [ ]:
def view_classify(img, ps):
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze(), cmap='gray')
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Probabilidad')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [ ]:

dataiter = iter(train_loader)
images, labels = dataiter.next()

images.resize_(10, 1, 784)

img_idx = 2
ps = model(images[img_idx,:])

img = images[img_idx]


In [ ]:
ps

In [ ]:
view_classify(img.view(1, 28, 28), ps)